#### a notebook to save all the data from the efficiency cuts for future use
Applying the cuts every time is just stupid

In [1]:
import sys,os,os.path

sys.path.append("../../")   # cite IC from parent directory
                            # NOTE if you can't import IC stuff, its because of the
                            # above line
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np
import tables as tb
import IC.invisible_cities.io.dst_io                           as     dstio
import IC.invisible_cities.io.mcinfo_io as mcio
from    IC.invisible_cities.core.core_functions   import shift_to_bin_centers

import scipy.special as special
from scipy.stats import skewnorm
from scipy.optimize import curve_fit

from scipy.integrate import quad

from Tl_studies.functions import isaura_analyse as isa

In [2]:
def apply_cuts_raw(tracks, e_low_cut = 0.05, fid_lower_z = 20, fid_upper_z = 1195, fid_r_lim = 472, e_lower = 1.55, e_upper = 1.65):
    '''
    Apply all relevant cuts and spit out dataframe at the end
    The most barebones version, no efficiency calcs, it just spits out the dataframe
    '''


    # Low energy tracks, make this function work better. It seems busted currently
    low_e_cut_tracks = isa.remove_low_E_events(tracks, energy_limit = e_low_cut)
  
    # make fiducial cuts
    fiducial_rel = isa.fiducial_track_cut_2(low_e_cut_tracks, lower_z = fid_lower_z, upper_z=fid_upper_z, r_lim = fid_r_lim, verbose = False)

    # one track cuts 
    one_track_rel = isa.one_track_cuts(fiducial_rel, verbose = False)

    # overlapping blob cut
    ovlp_rel = isa.overlapping_cuts(one_track_rel)


    # energy cuts
    ecut_rel = isa.energy_cuts(ovlp_rel, lower_e = e_lower, upper_e = e_upper)

    # return it
    return ecut_rel


In [3]:
folder_path = '../LPR_rerun_voxel/isaura_15x15x15mm/PORT_1a/prod/isaura/'

# load in tracks
data = isa.load_tracks(folder_path)

Warning! This method may take some time,
and works best with smaller datasets (like LPR).
100.00 %


In [4]:
display(data)

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
0,5080000,0,1.064505,157.128756,65,11348,1,368.675,163.675,334.439758,...,346.677116,388.298799,273.735214,442.301561,0.591051,0.013352,0.000000,13.800000,13.900000,14.429691
1,5080002,0,2.097495,339.919849,90,13467,2,-108.325,332.675,498.257599,...,651.573556,153.314470,367.920389,502.593622,0.375646,0.151995,0.000000,14.368421,14.888889,14.403850
2,5080002,1,0.008122,14.368421,2,70,2,27.675,410.675,618.963989,...,618.963989,39.001864,416.663406,620.861787,0.008122,0.008122,0.008122,14.368421,14.888889,14.403850
3,5080004,0,0.475855,33.046321,5,4467,2,19.675,-316.325,147.604263,...,154.720772,38.507312,-310.859095,153.875947,0.406043,0.388595,0.318784,13.333333,13.833333,14.510910
4,5080004,1,0.000826,0.000000,1,28,2,36.675,-259.325,154.847458,...,154.847458,38.559927,-256.434284,154.847458,0.000826,0.000826,0.000826,13.333333,13.833333,14.510910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189473,4060708,1,0.121964,20.069324,3,1689,3,187.675,341.675,304.032379,...,308.081912,202.815692,361.157542,308.356360,0.121667,0.121094,0.120797,13.333333,15.000000,14.789446
189474,4060708,2,0.033485,20.069324,2,457,3,147.675,360.675,386.938629,...,389.520812,163.627970,365.698728,389.654063,0.033368,0.032802,0.032685,13.333333,15.000000,14.789446
189475,4060710,0,0.572125,52.800000,21,7827,1,305.675,284.675,522.397217,...,529.509638,331.248510,294.552058,526.619440,0.242639,0.188812,0.000000,14.750000,13.200000,11.883850
189476,4060712,0,0.888838,93.553991,35,6761,2,-6.325,415.675,335.498718,...,346.560197,25.349946,477.323860,418.388821,0.759932,0.028161,0.000000,12.666667,14.200000,14.509995


In [5]:
cut_data = apply_cuts_raw(data)

/home/e78368jw/Documents/NEXT_CODE/Tl_studies/quick_notebooks/../../Tl_studies/functions/isaura_analyse.py:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remove_low_E['numb_of_tracks'] = remove_low_E['event'].map(event_counts)


In [6]:
display(cut_data)

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
37,5080036,0,1.596255,148.370606,57,11566,1,-98.325,308.675,260.970490,...,268.816186,-21.808474,407.837090,304.031722,0.682742,0.479417,0.0,14.166667,14.444444,12.045174
172,5080188,0,1.596243,135.904646,51,11926,1,156.675,107.675,734.490601,...,787.160369,246.035796,183.579923,740.719076,0.574430,0.397806,0.0,14.285714,14.000000,13.429004
471,5080496,0,1.594183,149.721509,60,12933,1,147.675,-166.325,322.920868,...,348.228080,153.085963,-124.132459,333.285062,0.456853,0.451467,0.0,14.300000,13.142857,13.105713
474,5080500,0,1.589982,205.359168,82,13395,1,-113.325,177.675,833.210754,...,839.223051,-65.638573,352.948549,859.177343,0.586061,0.246951,0.0,13.600000,14.307692,11.789322
488,5080512,0,1.598140,93.397310,40,13495,1,199.675,202.675,571.753174,...,581.635714,245.519546,214.330517,620.986981,0.702968,0.280580,0.0,14.000000,13.166667,14.151962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189310,4060532,0,1.592890,141.856792,68,12089,1,101.675,-359.325,833.190979,...,905.687916,120.576858,-264.555179,889.923837,0.086989,0.069762,0.0,13.666667,15.000000,13.189052
189323,4060546,0,1.595968,407.877953,84,10670,1,207.675,39.675,414.230682,...,600.959855,430.035414,48.042708,468.395088,0.202174,0.018846,0.0,14.941176,14.500000,14.777027
189326,4060550,0,1.624295,241.783804,97,11829,1,-46.325,-180.325,533.926270,...,717.327925,-0.542401,-159.900302,535.616629,0.439025,0.052087,0.0,13.714286,13.200000,14.534382
189370,4060606,0,1.594137,83.557401,51,14236,1,66.675,-1.325,499.301849,...,522.748218,134.966015,48.550782,506.152439,0.391017,0.156083,0.0,13.333333,14.750000,12.850258


In [8]:
# save cut data
cut_data.to_hdf('P_1A_10x10x10.h5', key = 'cut', mode = 'w')

In [9]:
test = pd.read_hdf('P_1A_10x10x10.h5')
display(test)
del test

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
184,5080188,0,1.596243,137.456298,113,11993,1,156.675,107.675,734.490601,...,789.447427,249.494267,184.766019,738.319437,0.511776,0.245868,0.0,10.000000,9.333333,9.592146
518,5080496,0,1.594183,154.743531,127,12870,1,147.675,-166.325,322.920868,...,348.181738,150.682122,-134.107884,329.234220,0.444847,0.443488,0.0,9.533333,9.200000,8.737142
606,5080582,0,1.595847,134.427703,165,12424,1,293.675,-43.325,683.036133,...,726.958593,344.200068,58.357780,714.376938,0.327431,0.169420,0.0,9.272727,9.615385,8.566711
674,5080676,0,1.594673,123.527479,117,12643,1,185.675,150.675,453.089264,...,466.743201,273.441110,157.500124,496.954274,0.612307,0.345949,0.0,9.416667,10.000000,9.292014
741,1740022,0,1.595029,246.160982,147,12240,1,186.675,-40.325,405.846893,...,498.364705,201.867704,58.954131,414.055713,0.492123,0.414989,0.0,10.000000,9.333333,9.967749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187495,4940584,0,1.598050,118.225238,144,12842,1,299.675,21.675,843.681885,...,909.920034,365.272391,95.558628,847.721470,0.475479,0.155838,0.0,9.100000,9.636364,8.917419
187630,4060092,0,1.611933,139.488392,138,11245,1,147.675,351.675,1015.907898,...,1023.885905,199.399870,392.845961,1111.865537,0.452174,0.263725,0.0,9.454545,9.100000,9.443298
187873,4060350,0,1.594714,193.786162,142,12044,1,43.675,21.675,412.145233,...,443.268241,95.273416,71.054266,541.965957,0.498915,0.433457,0.0,9.769231,9.000000,9.744980
187893,4060374,0,1.604224,110.101848,113,13741,1,231.675,227.675,516.088623,...,540.399202,301.620382,314.980297,538.400907,0.565798,0.366740,0.0,9.625000,9.300000,9.434460


In [7]:
folder_paths =  ['../LPR_rerun_voxel/isaura_15x15x15mm/PORT_1a/prod/isaura/',
                '../LPR_rerun_voxel/isaura_15x15x15mm/PORT_1b/prod/isaura/',
                '../LPR_rerun_voxel/isaura_15x15x15mm/PORT_2a/prod/isaura/',
                '../LPR_rerun_voxel/isaura_15x15x15mm/PORT_2b/prod/isaura/']
h5_file_names = ['P1A_15x15x15.h5',
                 'P1B_15x15x15.h5',
                 'P2A_15x15x15.h5',
                 'P2B_15x15x15.h5']


for i in range(len(folder_paths)):
    print(h5_file_names[i])
    print("")
    # load data
    data = isa.load_tracks(folder_paths[i])

    # cut data
    cut_data = apply_cuts_raw(data)

    # save data
    cut_data.to_hdf(h5_file_names[i], key = 'cut', mode = 'w')

    # cleanup
    del data
    del cut_data

P1A_15x15x15.h5

Warning! This method may take some time,
and works best with smaller datasets (like LPR).
100.00 %


/home/e78368jw/Documents/NEXT_CODE/Tl_studies/quick_notebooks/../../Tl_studies/functions/isaura_analyse.py:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remove_low_E['numb_of_tracks'] = remove_low_E['event'].map(event_counts)


P1B_15x15x15.h5

Warning! This method may take some time,
and works best with smaller datasets (like LPR).
100.00 %
P2A_15x15x15.h5

Warning! This method may take some time,
and works best with smaller datasets (like LPR).
100.00 %
P2B_15x15x15.h5

Warning! This method may take some time,
and works best with smaller datasets (like LPR).
100.00 %


In [9]:
# proof of concept
proof = pd.read_hdf('P1A_15x15x15.h5')
display(proof.head(20))

,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
37,5080036,0,1.596255,148.370606,57,11566,1,-98.325,308.675,260.970490,...,268.816186,-21.808474,407.837090,304.031722,0.682742,0.479417,0.0,14.166667,14.444444,12.045174
172,5080188,0,1.596243,135.904646,51,11926,1,156.675,107.675,734.490601,...,787.160369,246.035796,183.579923,740.719076,0.574430,0.397806,0.0,14.285714,14.000000,13.429004
471,5080496,0,1.594183,149.721509,60,12933,1,147.675,-166.325,322.920868,...,348.228080,153.085963,-124.132459,333.285062,0.456853,0.451467,0.0,14.300000,13.142857,13.105713
474,5080500,0,1.589982,205.359168,82,13395,1,-113.325,177.675,833.210754,...,839.223051,-65.638573,352.948549,859.177343,0.586061,0.246951,0.0,13.600000,14.307692,11.789322
488,5080512,0,1.598140,93.397310,40,13495,1,199.675,202.675,571.753174,...,581.635714,245.519546,214.330517,620.986981,0.702968,0.280580,0.0,14.000000,13.166667,14.151962
525,5080546,0,1.624835,135.647936,73,13102,1,274.675,205.675,601.143860,...,653.039899,281.034126,279.485456,602.943345,0.478832,0.054695,0.0,13.857143,14.125000,12.588403
560,5080582,0,1.595847,127.273875,67,12444,1,293.675,-39.325,683.036133,...,725.444528,343.848974,61.787506,705.178278,0.332589,0.162293,0.0,14.571429,13.888889,12.850067
632,5080668,0,1.592118,104.519054,46,11666,1,-47.325,148.675,664.138794,...,674.760209,8.766727,159.889750,742.899318,0.578547,0.538527,0.0,14.600000,14.250000,14.163818
637,5080676,0,1.594673,136.079427,62,12806,1,177.675,150.675,453.089264,...,463.562800,276.599374,159.072824,496.926709,0.608535,0.344964,0.0,14.125000,15.000000,13.008820
722,1740022,0,1.595029,252.199785,75,12399,1,186.675,-40.325,405.846893,...,499.186960,198.556032,62.293305,416.669783,0.496267,0.364702,0.0,15.000000,14.000000,14.239641
